# Úkol 3b

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shippy/czechitas-ai-data/blob/main/notebooks/assignment-03b.ipynb)

Pokud nebudete vědět, nezapomeňte se zeptat [pomocníčka pro tento úkol](https://chatgpt.com/g/g-67cab661ae5c8191b0d8419c76d3959b-czechitas-ai-in-data-analytics-2025-03)!

## Analýza článků v časopisu Guardian

V tomto notebooku se budeme věnovat zpracování nestrukturovaných dat, které jsou náhodně zvolenou podmnožinou [většího datasetu z Kaggle](https://www.kaggle.com/datasets/adityakharosekar2/guardian-news-articles?resource=download).

Nejprve nastavíme klienta pro OpenAI API. Toto nastavení se bude lišit v závislosti na tom, zda používáte Google Colab nebo spouštíte notebook na vlastním počítači, ale pokud následující dvě buňky proběhnou bez chyby, jste připraveni pokračovat.

In [1]:
try:
    from google.colab import userdata
    _secret = userdata.get("OPENAI_API_KEY")
    %pip install pandas instructor openai python-dotenv rich
    !curl https://raw.githubusercontent.com/shippy/czechitas-ai-data/refs/heads/main/notebooks/guardian_100.csv
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("OPENAI_API_KEY")

In [ ]:
import instructor
from openai import OpenAI
from rich import print

try:
    if not _secret:
        raise ValueError("API klíč nebyl nastaven!")
except NameError:
    print("Nastavte si API klíč v proměnné OPENAI_API_KEY a znovu spusťte *celý* notebook včetně předchozí buňky")

_client = OpenAI(api_key=_secret)
client = instructor.from_openai(_client)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you? Respond with an emotion."},
    ],
    model="gpt-4o",
    response_model=str,
)
print(test)


## Čtení dat

In [ ]:
import pandas as pd

data = pd.read_csv('guardian_100.csv')
print(data.shape)
data.head()


## Úkoly

1. Vytvořte a implementujte prompt, který shrne každý z článků do jedné věty.
2. Vytvořte prompt, který z textu článku vytvoří strukturovaný výstup, ve kterém bude uvedeno, jakých lidí/entit se týká, do jaké z předdefinovaných kategorií (`data['sectionName'].unique().tolist()`) spadá, a jak urgentní popisuje situaci. Uložte tuto strukturu do nového sloupce `data['structured_output']`.
3. Udělejte graf častosti výskytu jednotlivých kategorií a jednotlivých entit. (Fantazii se meze nekladou - prodiskutujte to s pomocníčkem.)
4. Porovnejte výsledky klasifikace s původním sloupcem `data['sectionName']`. Jak velké úspěšnosti jste dosáhly?

## Ukázka

In [ ]:
example = data[0:2]
outputs = []
for i, row in example.iterrows():
    print('Headline: ', row['webTitle'])
    output = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes news articles down to exactly one sentence."},
            {"role": "user", "content": row['bodyContent']},
        ],
        model="gpt-4o",
        response_model=str,
    )
    print('One-sentence summary: ', output)
    outputs.append(output)


## Úkol 1

Shrňte každý článek do jedné věty.

In [ ]:
summaries = []
for idx, row in data.iterrows():
    summary = ...  # tohle nahraďte správným kódem
    summaries.append(summary)

# data['oneSentenceSummary'] = summaries

## Úkol 2

Vytvořte prompt, který z textu článku vytvoří strukturovaný výstup, ve kterém bude uvedeno, jakých lidí/entit se týká, do jaké z předdefinovaných kategorií (`data['sectionName'].unique().tolist()`) spadá, a jak urgentní popisuje situaci. Uložte tuto strukturu do nového sloupce `data['structured_output']`.

In [5]:
from enum import Enum
from pydantic import BaseModel

def create_enum(name: str, values: list[str]) -> Enum:
    return Enum(name, {v.upper(): v for v in values})

CategoryEnum = create_enum('CategoryEnum', data['sectionName'].unique().tolist())

class ArticleSummary(BaseModel):
    summary: str
    entities: list[str]
    category: CategoryEnum
    urgency: int


outputs = []

for idx, row in data.iterrows():
    output = ...  # tohle nahraďte správným kódem
    outputs.append(output)

# data['structured_output'] = outputs


